In [4]:
from datasets import load_dataset

ds = load_dataset("cbasu/Med-EASi")

print(ds)
print("Train dataset sample: ")
ds['train'][0]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1397 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/196 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/300 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Expert', 'Simple', 'Annotation', 'sim', 'sentence_sim', 'compression', 'expert_fk_grade', 'expert_ari', 'layman_fk_grade', 'layman_ari', 'umls_expert', 'umls_layman', 'expert_terms', 'layman_terms', 'idx'],
        num_rows: 1397
    })
    validation: Dataset({
        features: ['Expert', 'Simple', 'Annotation', 'sim', 'sentence_sim', 'compression', 'expert_fk_grade', 'expert_ari', 'layman_fk_grade', 'layman_ari', 'umls_expert', 'umls_layman', 'expert_terms', 'layman_terms', 'idx'],
        num_rows: 196
    })
    test: Dataset({
        features: ['Expert', 'Simple', 'Annotation', 'sim', 'sentence_sim', 'compression', 'expert_fk_grade', 'expert_ari', 'layman_fk_grade', 'layman_ari', 'umls_expert', 'umls_layman', 'expert_terms', 'layman_terms', 'idx'],
        num_rows: 300
    })
})
Train dataset sample: 


{'Expert': '75-90 % of the affected people have mild intellectual disability.',
 'Simple': "People with syndromic intellectual disabilities may have a `` typical look. ''",
 'Annotation': "<del>75-90 % of the</del> <rep>affected people  have mild intellectual disability.<by>People with syndromic intellectual disabilities</rep> <ins>may have a `` typical look. ''</ins>",
 'sim': 0.48951049,
 'sentence_sim': 0.639872432,
 'compression': 1.2,
 'expert_fk_grade': 12.7,
 'expert_ari': 12.4,
 'layman_fk_grade': 13.1,
 'layman_ari': 15.1,
 'umls_expert': "[[{'start': 41, 'end': 64, 'ngram': 'intellectual disability', 'term': 'intellectual disability', 'cui': 'C3714756', 'similarity': 1.0, 'semtypes': {'T048'}, 'preferred': 1, 'preferred_term': None}, {'start': 41, 'end': 64, 'ngram': 'intellectual disability', 'term': 'Intellectual disability', 'cui': 'C3714756', 'similarity': 0.9090909090909091, 'semtypes': {'T048'}, 'preferred': 0, 'preferred_term': None}, {'start': 41, 'end': 64, 'ngram': 

In [5]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")


def preprocess_and_tokenizer(batch):
    inputs = batch['Expert']
    targets = batch['Simple']

    model_inputs = tokenizer(inputs, truncation=True, max_length=256)
    labels = tokenizer(text_target=targets, truncation=True, max_length=256)["input_ids"]

    model_inputs["labels"] = labels
    return model_inputs


col_names = ds["train"].column_names
tokenized_ds = ds.map(preprocess_and_tokenizer, batched=True, remove_columns=col_names)

tokenized_ds

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/1397 [00:00<?, ? examples/s]

Map:   0%|          | 0/196 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1397
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 196
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 300
    })
})

In [6]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
import wandb

model_name = "google/flan-t5-base"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

wandb.init(project="Med-EASE", name="flan_t5_base_baseline")

training_args = Seq2SeqTrainingArguments(
    output_dir="checkpoints/flan_t5_baseline",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=3e-5,
    num_train_epochs=5,
    warmup_steps=1000,
    logging_steps=50,
    save_steps=500,
    eval_steps=500,
    load_best_model_at_end=True,
    tpu_num_cores=8,
    predict_with_generate=True,
    save_total_limit=2,
)


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)


trainer.train()

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sriyamed1 (sriyamed1-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/tmp/ipython-input-1921258865.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,1.593800,1.883344
2,1.559400,1.832589
3,1.470800,1.795272
4,1.338600,1.776446
5,1.287000,1.767708


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=875, training_loss=1.4866534641810827, metrics={'train_runtime': 482.0605, 'train_samples_per_second': 14.49, 'train_steps_per_second': 1.815, 'total_flos': 691261209879552.0, 'train_loss': 1.4866534641810827, 'epoch': 5.0})

In [7]:
wandb.finish()

eval/loss,█▅▃▂▁
eval/runtime,█▁▆▄▅
eval/samples_per_second,▁█▃▅▄
eval/steps_per_second,▁█▃▅▄
train/epoch,▁▁▂▂▂▃▃▄▄▄▄▅▅▅▆▆▇▇▇▇███
train/global_step,▁▁▂▂▂▃▃▄▄▄▄▅▅▅▆▆▇▇▇▇███
train/grad_norm,█▅▆▄▄▂▆▂▃▃▃▄▁▄▄▁▃
train/learning_rate,▁▁▂▂▃▃▄▄▄▅▅▆▆▇▇██
train/loss,█▇▆▅▅▅▅▃▄▄▃▃▄▂▂▂▁
eval/loss,1.76771
eval/runtime,3.3153


In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
test_dataset = ds["test"]
def generate_predictions(batch):
    inputs = tokenizer(
        batch["Expert"],
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    outputs = model.generate(
        input_ids=inputs["input_ids"].to(model.device),
        attention_mask=inputs["attention_mask"].to(model.device),
        max_length=180, 
        do_sample=True,
        top_k=70,
        top_p=0.99,
        temperature=0.8
    )

    batch["pred"] = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return batch

In [9]:
predictions = test_dataset.map(generate_predictions, batched=True, batch_size=8)


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [11]:
!pip install evaluate
!pip install sacremoses sacrebleu
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=73eda0d68ba0ea6f0e7fa9bf29a26634a6d5dadc567f08615ecad6d9133c27e0
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [12]:
import evaluate

sari_metric = evaluate.load("sari")
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
sari_score = sari_metric.compute(
    sources=predictions["Expert"],
    predictions=predictions["pred"],
    references=[[ref] for ref in predictions["Simple"]]
)

bleu_score = bleu_metric.compute(
    predictions=predictions["pred"],
    references=[[ref] for ref in predictions["Simple"]]
)

rouge_score = rouge_metric.compute(
    predictions=predictions["pred"],
    references=predictions["Simple"]
)
print("SARI:", sari_score)
print("BLEU:", bleu_score)
print("ROUGE:", rouge_score)


SARI: {'sari': 42.80723964712648}
BLEU: {'bleu': 0.2553565447465041, 'precisions': [0.5244569233042707, 0.329055203340034, 0.26475356679636836, 0.22559209405350145], 'brevity_penalty': 0.8014213778567862, 'length_ratio': 0.8187537810042347, 'translation_length': 6767, 'reference_length': 8265}
ROUGE: {'rouge1': np.float64(0.4685610930475161), 'rouge2': np.float64(0.3154952798904501), 'rougeL': np.float64(0.43238045708344913), 'rougeLsum': np.float64(0.43168646793943)}


In [ ]:
import pandas as pd

df = pd.DataFrame({
    "expert": predictions["Expert"],
    "reference_simple": predictions["Simple"],
    "model_simple": predictions["pred"]
})

df.to_csv("model_outputs.csv", index=False)
